In [1]:

import sys
sys.path.append("./")
sys.path.append("../")

import numpy as np
import pandas as pd
from utils.data_utils import stamp_to_date
from utils.data_utils import date_to_week


def data_preprocess(data, time_col, back_time, dtypes_dict):
    """
    数据预处理函数
    :param data: 待处理的数据
    :param time_col: 回溯依据的时间列名称
    :param back_time: 特征计算时间，datetime.datetime时间格式
    :param dtypes_dict: 指定列字段类型的字典，如{'col1':int}
    :return: 清洗完成的数据
    """
    # 删除time_col为空的行
    data = data[~data[time_col].isin(['nan', np.nan, 'NAN', 'null', 'NULL', 'Null'])]
    # 将时间列的时间戳转为日期格式
    data[time_col] = data[time_col].apply(stamp_to_date)
    # 过滤订单创建时间在back_time之后的数据，避免特征穿越
    data = data[data[time_col] <= back_time]
    # 删除整条缺失的数据
    data.dropna(how='all', inplace=True)
    # 空字符串替换为np.nan
    data.replace('', np.nan, inplace=True)
    # 单个字段缺失填充为0
    data.fillna(0, inplace=True)
    # 去重
    data.drop_duplicates(keep='first', inplace=True)
    # 字段格式转换
    data = data.astype(dtypes_dict)
    # 补充字段
    data['create_time_week'] = data[time_col].apply(date_to_week)
    data['is_weekend'] = data['create_time_week'].apply(lambda x: 1 if x > 5 else 0)

    return data

In [2]:
orders = pd.read_excel('data/order_data.xlsx')

In [8]:
raw_data = pd.DataFrame(eval(orders['data'][1]))

In [10]:
data_processed = data_preprocess(raw_data, time_col='create_time',
                                     back_time='2020-12-14',
                                     dtypes_dict={'has_overdue': int,
                                                  'application_term': float,
                                                  'application_amount': float})

In [13]:
import sys

sys.path.append("./")
sys.path.append("../")

# 根据业务逻辑自动生成用户历史订单特征
import pandas as pd
import numpy as np
from dateutil.parser import parse
from utils.data_utils import stamp_to_date
from chapter3.ch3_00_order_data_preprocess import data_preprocess

func_trans = {'sum': np.sum,
              'mean': np.mean,
              'cnt': np.size,
              'max': np.max,
              'min': np.min,
              'std': np.std,
              }


def apply_func(f, *args):
    return f(*args)


def rfm_cut(data, time_col, back_time, type_dict, comp_dict, time_arr, fea_prefix='f'):
    """
    基于RFM思想切分数据，生成特征
    :param DataFrame data: 待切分的数据，时间列为create_time(timestamp)，距今天数列为gap_days
    :param str time_col: 回溯依据的时间列名称
    :param datetime.datetime back_time: 回溯时间点，datetime.datetime时间格式
    :param dict type_dict: 类别变量，以及其对应的取值类别，用于划分数据，类别列名必须在data中
    :param dict comp_dict: 指定计算字段以及对该字段采用的计算方法, 计算变量名必须在data中
    :param list time_arr: 切分时间列表(近N天)
    :param fea_prefix: 特征前缀
    :return dict: 特征
    """
    data[time_col] = data[time_col].apply(stamp_to_date)
    # 业务时间距back_time天数
    data['gap_days'] = data[time_col].apply(lambda x: (back_time - x).days)

    res_feas = {}
    for col_time in time_arr:
        for col_comp in comp_dict.keys():
            for type_k, type_v in type_dict.items():
                # 按类别和时间维度切分,筛选数据
                for item in type_v:
                    data_cut = data[(data['gap_days'] < col_time) & (data[type_k] == item)]
                    for func_k in comp_dict[col_comp]:
                        func_v = func_trans.get(func_k, np.size)
                        # 对筛选出的数据, 在各统计指标上做聚合操作生成特征
                        fea_name = '%s_%s_%s_%s_%s' % (
                            fea_prefix, col_time, '%s_%s' % (type_k, item), col_comp, func_k)
                        if data_cut.empty:
                            res_feas[fea_name] = np.nan
                        else:
                            res_feas[fea_name] = apply_func(func_v, data_cut[col_comp])
    return res_feas


def gen_order_feature_auto(raw_data, time_col, back_time, dtypes_dict, type_dict, comp_dict, time_arr,
                           fea_prefix='f'):
    """
    基于RFM切分，自动生成订单特征
    :param pd.DataFrame raw_data: 原始数据
    :param str time_col: 回溯依据的时间列名称
    :param str back_time: 回溯时间点，字符串格式
    :param dict dtypes_dict: 指定列字段类型的字典，如{'col1':int}
    :param list time_arr: 切分时间列表(近N天)
    :param dict type_dict: 类别变量，以及其对应的取值类别，用于划分数据，类别列名必须在data中
    :param dict comp_dict: 指定计算字段以及对该字段采用的计算方法,计算变量名必须在data中
    :param fea_prefix: 特征前缀
    :return: res_feas 最终生成的特征
    """
    if raw_data.empty:
        return {}
    back_time = parse(str(back_time))

    order_df = data_preprocess(raw_data, time_col=time_col, back_time=back_time, dtypes_dict=dtypes_dict)
    if order_df.empty:
        return {}

    # 特征衍生：使用rfm切分
    res_feas = rfm_cut(order_df, time_col, back_time, type_dict, comp_dict, time_arr, fea_prefix)
    return res_feas

In [17]:
# 原始数据读入
orders = pd.read_excel('data/order_data.xlsx')
# 取一个用户的历史订单数据
raw_orders = pd.DataFrame(eval(orders['data'][1]))

type_dict_param = {
        'has_overdue': [0, 1],
        'is_weekend': [0, 1]
    }
    # 计算字段及其计算函数
comp_dict_param = {
        'order_no': ['cnt'],
        'application_amount': ['sum', 'mean', 'max', 'min']
    }
time_cut = [30, 90, 180, 365]

cols_dtypes_dict = {'has_overdue': int, 'application_term': float, 'application_amount': float}

    # 根据业务逻辑生成用户历史订单特征
features_auto = gen_order_feature_auto(raw_orders, 'create_time', '2020-12-14', cols_dtypes_dict,
                                           type_dict_param, comp_dict_param, time_cut)
print("特征维度: ", len(features_auto.keys()))
print(features_auto)

    # 批量生成特征
feature_dict = {}
for i, row in orders.iterrows():
        feature_dict[i] = gen_order_feature_auto(pd.DataFrame(eval(row['data'])), 'create_time', row['back_time'],cols_dtypes_dict, type_dict_param, comp_dict_param, time_cut,
                                                 'order_auto')
feature_df_auto = pd.DataFrame(feature_dict).T

特征维度:  80
{'f_30_has_overdue_0_order_no_cnt': 11, 'f_30_has_overdue_1_order_no_cnt': 1, 'f_30_is_weekend_0_order_no_cnt': 11, 'f_30_is_weekend_1_order_no_cnt': 1, 'f_30_has_overdue_0_application_amount_sum': 5170000.0, 'f_30_has_overdue_0_application_amount_mean': 470000.0, 'f_30_has_overdue_0_application_amount_max': 850000.0, 'f_30_has_overdue_0_application_amount_min': 160000.0, 'f_30_has_overdue_1_application_amount_sum': 850000.0, 'f_30_has_overdue_1_application_amount_mean': 850000.0, 'f_30_has_overdue_1_application_amount_max': 850000.0, 'f_30_has_overdue_1_application_amount_min': 850000.0, 'f_30_is_weekend_0_application_amount_sum': 5820000.0, 'f_30_is_weekend_0_application_amount_mean': 529090.9090909091, 'f_30_is_weekend_0_application_amount_max': 850000.0, 'f_30_is_weekend_0_application_amount_min': 160000.0, 'f_30_is_weekend_1_application_amount_sum': 200000.0, 'f_30_is_weekend_1_application_amount_mean': 200000.0, 'f_30_is_weekend_1_application_amount_max': 200000.0, 'f_3

In [18]:
feature_df_auto

,order_auto_30_has_overdue_0_order_no_cnt,order_auto_30_has_overdue_1_order_no_cnt,order_auto_30_is_weekend_0_order_no_cnt,order_auto_30_is_weekend_1_order_no_cnt,order_auto_30_has_overdue_0_application_amount_sum,order_auto_30_has_overdue_0_application_amount_mean,order_auto_30_has_overdue_0_application_amount_max,order_auto_30_has_overdue_0_application_amount_min,order_auto_30_has_overdue_1_application_amount_sum,order_auto_30_has_overdue_1_application_amount_mean,...,order_auto_365_has_overdue_1_application_amount_max,order_auto_365_has_overdue_1_application_amount_min,order_auto_365_is_weekend_0_application_amount_sum,order_auto_365_is_weekend_0_application_amount_mean,order_auto_365_is_weekend_0_application_amount_max,order_auto_365_is_weekend_0_application_amount_min,order_auto_365_is_weekend_1_application_amount_sum,order_auto_365_is_weekend_1_application_amount_mean,order_auto_365_is_weekend_1_application_amount_max,order_auto_365_is_weekend_1_application_amount_min
0,5.0,1.0,3.0,3.0,2910000.0,582000.000000,800000.0,460000.0,740000.0,740000.000000,...,740000.0,740000.0,1710000.0,570000.000000,740000.0,460000.0,1940000.0,646666.666667,800000.0,460000.0
1,13.0,3.0,15.0,1.0,6170000.0,474615.384615,850000.0,160000.0,2220000.0,740000.000000,...,850000.0,620000.0,8190000.0,546000.000000,850000.0,160000.0,200000.0,200000.000000,200000.0,200000.0
2,11.0,3.0,11.0,3.0,4010000.0,364545.454545,510000.0,160000.0,1670000.0,556666.666667,...,740000.0,420000.0,4620000.0,420000.000000,740000.0,160000.0,1060000.0,353333.333333,500000.0,160000.0
3,13.0,2.0,15.0,NaN,5800000.0,446153.846154,680000.0,200000.0,1020000.0,510000.000000,...,510000.0,510000.0,6820000.0,454666.666667,680000.0,200000.0,NaN,NaN,NaN,NaN
4,20.0,2.0,22.0,NaN,9690000.0,484500.000000,800000.0,210000.0,1200000.0,600000.000000,...,800000.0,400000.0,10890000.0,495000.000000,800000.0,210000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,10.0,3.0,11.0,2.0,4080000.0,408000.000000,620000.0,120000.0,1650000.0,550000.000000,...,630000.0,510000.0,4650000.0,422727.272727,630000.0,120000.0,1080000.0,540000.000000,620000.0,460000.0
196,1.0,2.0,3.0,NaN,460000.0,460000.000000,460000.0,460000.0,1530000.0,765000.000000,...,850000.0,680000.0,1990000.0,663333.333333,850000.0,460000.0,NaN,NaN,NaN,NaN
197,6.0,4.0,8.0,2.0,2980000.0,496666.666667,560000.0,400000.0,2670000.0,667500.000000,...,750000.0,550000.0,4340000.0,542500.000000,740000.0,400000.0,1310000.0,655000.000000,750000.0,560000.0
198,2.0,2.0,2.0,2.0,970000.0,485000.000000,510000.0,460000.0,1300000.0,650000.000000,...,680000.0,620000.0,1300000.0,650000.000000,680000.0,620000.0,970000.0,485000.000000,510000.0,460000.0
